In [1]:
import praw
import pandas as pd
import numpy as np
import json
from datetime import datetime
from kafka import KafkaProducer
from confluent_kafka import Producer
from confluent_kafka import Consumer
# import psycopg2

In [34]:
# Kafka configuration
bootstrap_servers = "localhost:9092"
kafka_topic = "reddit"
producer = Producer({'bootstrap.servers': bootstrap_servers})
# producer = Producer(bootstrap_servers=bootstrap_servers, request_timeout_ms=1200000)

In [36]:
from confluent_kafka import Consumer

# Configuration du consommateur
config = {'bootstrap.servers': bootstrap_servers, 'group.id': 'Xmen', 'auto.offset.reset': 'earliest'}

# Création du consommateur
consumer = Consumer(config)

# S'abonner à un ou plusieurs topics
topics = ['reddit_postgres']
consumer.subscribe(topics)

In [37]:
import psycopg2

In [ ]:
from nltk.corpus import stopwords
import re

In [ ]:
def clean_string(string, stopwords):
    clean_string = re.sub(r'[^\w\s]', '', string)
    clean_strings = clean_string.lower().split(" ")
    cleans = [x for x in clean_strings if not x in stopwords]
    cleaned_string = " ".join(cleans)
    return cleaned_string

In [ ]:
stops = stopwords.words('english')
stops.extend(["wa", "hi", "a", "thi", "would", "had", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves', "could", "thus"])

In [ ]:
import joblib
model = joblib.load("pipe.pkl")

# SQL statement to create the "tweets" table
create_table_query = """
CREATE TABLE IF NOT EXISTS tweets (
    id SERIAL PRIMARY KEY,
    tweet_text TEXT,
    tweet_date TIMESTAMP,
    topic TEXT,
    label TEXT,
    clean TEXT
);
"""
# Execute the table creation query
postgres_cursor.execute('DROP TABLE tweets')
postgres_cursor.execute(create_table_query)
postgres_conn.commit()

# Consommer les messages
while True:
    message = consumer.poll(1.0)  # Attendre pendant 1 seconde au maximum
    
    if message is None:
        continue
    
    if message.error():
        print(f"Erreur lors de la consommation : {message.error()}")
        continue
    
    # Traitement du message
    print("Message reçu : {}".format(message.value().decode('utf-8')))
    

    tweet = eval(message.value().decode('utf-8'))

    tweet["label"] = model.predict([tweet["value"]])[0]
    tweet["clean"] = clean_string(tweet["value"], stops)
    # Configure PostgreSQL connection
    postgres_connection_params = {
        "host": 'localhost',
        # "port": postgres_port,
        "database": 'reddit',
        "user": 'postgres',
        "password": 'elodie98',
    }
    postgres_conn = psycopg2.connect(**postgres_connection_params)
    postgres_cursor = postgres_conn.cursor()

    tweet_text = tweet["value"].encode('utf-8', errors='ignore').decode('utf-8')
    tweet_date = tweet["date"]
    topic = tweet["topic"]
    label = tweet["label"]
    clean = tweet["clean"]
    
    postgres_insert_query = "INSERT INTO tweets (tweet_text, tweet_date, topic, label, clean) VALUES (%s,%s,%s,%s,%s)"
    record_to_insert = (tweet_text, tweet_date, topic, label, clean)
    print(record_to_insert)
    postgres_cursor.execute(postgres_insert_query, record_to_insert)
    postgres_conn.commit()
    # Close the PostgreSQL connection
    postgres_cursor.close()
    postgres_conn.close()
    
# Fermeture du consommateur
consumer.close()